# Setup

In [ ]:
!pip list

## Installations

In [ ]:
%pip install -r "requirements.txt"

In [ ]:
%pip uninstall -y triton

## Imports

In [1]:
import pandas as pd
import gffpandas.gffpandas as gffpd
from py2neo import Graph, Node, Relationship, cypher
from bisect import bisect_left as bisect

In [2]:
from typing import Sequence, Callable
from random import Random, randrange
import Levenshtein as lev
from multiprocessing.pool import ThreadPool as Pool

In [3]:
import os
import torch
from transformers import AutoTokenizer, AutoModel

/home/daftylooper/Desktop/Capstone/code/finetune-chatgpt-go/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import transformers
transformers.__version__

'4.29.2'

In [5]:
def thread_starmap(arg_list, *, threads, progress_step=255):
  def progress(f):
    def inner(i, args):
      if not i&progress_step: print('Invocation', i)
      return f(*args)

  def inner(fn):
    with Pool(threads) as p:
      print('Running map...')
      p.starmap(progress(fn), enumerate(arg_list))
      print('Map complete. Destroying pool')
    print('Pool complete.')
    return fn
  return inner

## Connection

In [7]:
import os
from dotenv import load_dotenv
load_dotenv()

graph = Graph("neo4j+s://"+os.getenv("neo4j_connection_url"), auth=(os.getenv("neo4j_user"), os.getenv("neo4j_password")))

# Check connection
graph.run("RETURN 1")

ServiceUnavailable: Cannot connect to any known routers

# Main Graph

In [ ]:
def add_triple_to_graph(i, triple):

  if not i&255: print('Triple number', i)
      
  h, r, t = triple
  # Create or match the nodes, 'name' is a property
  head_node = Node("Entity", name=h)
  tail_node = Node("Entity", name=t)

  # Merge ensures nodes are only created if they don't already exist
  graph.merge(head_node, "Entity", "name")
  graph.merge(tail_node, "Entity", "name")

  # Create the relationship (head_node)-[relation]->(tail_node)    
  relationship = Relationship(head_node, r, tail_node)

  # Merge the relationship into the graph
  graph.merge(relationship)

## Create Sequence and Identity nodes + relations

In [ ]:
import csv
exon_data = []
path = "./data/exons/exon_sequences.csv"
hgnc = set()
labels = ["gene_id","transcript_id","hgnc_id","gene_type","gene_name","exon_id","protein_id","sequence"]
count = 0
with open(path, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        # hgnc.add(tuple(row))
        exon_data.append(dict(zip(labels, row)))

# print(f"number of unique hgnc ids {len(hgnc)}")
exon_data = exon_data[1:]
print(f"pulled {len(exon_data)} sequences!")
exon_data = [kv for kv in exon_data if len(kv)==len(labels)]
print(f"new length {len(exon_data)}")

In [ ]:
# maybe make a decorator for the thread pool thingi?

# TODO: ADD GO ref ids to Identity nodes later
seq_node_id_node_query = '''
MERGE (s:Sequence {sequence: $sequence, exon_id: $exon_id, protein_id: $protein_id})-[:MAPS_TO]->(i:Identity {gene_id: $gene_id, hgnc_id: $hgnc_id, transcript_id: $transcript_id});
'''
def make_seq_node_with_id_node(i, d):
    # arr is of type - [ensembl_id, gene_type, gene_name, hgnc_id, seq]
    if not i&127: print(f"Processed {i} nodes!")
    graph.run(seq_node_id_node_query, **d)
    

In [ ]:
sequence_identity_node_query = '''
MERGE (s:Sequence {sequence: $sequence, exon_id: $exon_id, protein_id: $protein_id})
MERGE (t:Transcript {transcript_id: $transcript_id})
MERGE (g:Gene {gene_id: $gene_id, hgnc_id: $hgnc_id})
MERGE (s)-[:MAPS_TO]->(t)
MERGE (t)-[:PART_OF]->(g);
'''
# identity_node_query = '''
# MERGE (i:Identity {gene_id: $gene_id, hgnc_id: $hgnc_id, transcript_id: $transcript_id})
# MATCH (e:Sequence {exon_id: $exon_id}), (i:Identity {gene_id: $gene_id})
# MERGE (e)-[:MAPS_TO]->(i);
# '''
# sequence_identity_relation_query = '''
# MATCH (e:Sequence {exon_id: $exon_id}), (i:Identity {gene_id: $gene_id})
# MERGE (e)-[:MAPS_TO]->(i);
# '''
count = 0
total = len(exon_data)
def make_seq_node_with_id_node(i, d):
    global count, total
    # arr is of type - [ensembl_id, gene_type, gene_name, hgnc_id, seq]
    if not count&127: print(f"Processed {count}/{total} nodes!")
    graph.run(sequence_identity_node_query, **d)
    count+=1

In [ ]:
print('Creating pool')
with Pool(100) as p:
  print('Running map...')
  p.starmap(make_seq_node_with_id_node, enumerate(exon_data))
  print('Queries complete. Destroying pool')
print('Pool complete.')

## Map Ensembl ID to Go

In [ ]:
import csv

labels = ["gene_id", "go"]
path = "./data/go/ensembl_go_protein_coding.csv"
go_data = []
go = set()
with open(path, "r") as f:
    reader = csv.reader(f)
    for row in reader:
        go_data.append(dict(zip(labels, row)))
        # go.add(row[1])

go_data = go_data[1:]
print(f"pulled {len(go_data)} records!")
# print(f"unique go ids: {len(go)}")

In [ ]:
gene_go_mapper_query = '''
MATCH (s: Gene {gene_id: $gene_id})
MERGE (go: Go {go_id: $go})
MERGE (s)-[:HAS_GO]->(go);
'''
count = 0
total = len(go_data)
def map_gene_to_go(i, d):
    global count, total
    if not count&127: print(f"Processed {count}/{total} nodes!")
    graph.run(gene_go_mapper_query, **d)
    count+=1

In [ ]:
print('Creating pool')
with Pool(100) as p:
  print('Running map...')
  p.starmap(map_gene_to_go, enumerate(go_data))
  print('Queries complete. Destroying pool')
print('Pool complete.')

In [ ]:
import csv

go_triples = []
path = "./data/go/go_triples_protein_coding.csv"
labels = ["go_id", "relation_type", "property"]

with open(path, "r") as f:
    reader = csv.reader(f)
    for row in reader:
        go_triples.append(dict(zip(labels, row)))

go_triples = go_triples[1:]
print(go_triples[0])
print(go_triples[3])
print(f"pulled {len(go_triples)} go triples!")

In [ ]:
count = 0
total = len(go_triples)
add_go_triple_query = '''
MATCH (go:Go {go_id: $go_id})
MERGE (gop:GoProperty {property: $property, type: $relation_type})
'''
relation = "MERGE (go)-[:{relation_type}]->(gop)"

def add_go_triple_to_graph(i, triple):
    global count, total
    if not count&127 and count!=0: print(f"processed {count}/{total} triples!")
    r = relation.format(relation_type=triple["relation_type"])
    q = add_go_triple_query+r
    graph.run(q, **triple)
    count+=1

In [ ]:
print('Creating pool')
with Pool(100) as p:
  print('Running map...')
  p.starmap(add_go_triple_to_graph, enumerate(go_triples))
  print('Queries complete. Destroying pool')
print('Pool complete.')

## UniprotKB to KG

In [ ]:
# read every line, csv file
# form a dict of every line( key=header, value=whatever we read)
# create a string.format() to add these k-v's into query
# query creates a node from the sequence with relation type being the header
# run query

import csv

path = "./data/uniprotkb/uniprotkb_protein_coding.csv"
t = []
with open(path, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        t.append(row)

header = t[0]
t = t[1:]
distinct = set()
uniprot = []

print(f"{len(header)} headers read")
print(f"{len(t)} uniprot data slices")
for row in t:
    distinct.add(tuple(row))
for row in distinct:
    uniprot.append(dict(zip(header, row)))

print(f"found {len(uniprot)} unique sequences!")

print(uniprot[0])

In [ ]:
create_uniprot_index = '''
MATCH (s:Sequence {protein_id: $protein_id})
MERGE (u:Uniprot {uniprot_id: $uniprot_id})
MERGE (s)-[:HAS_UNIPROT]->(u)
'''

create_uniprot_nodes = '''
MATCH (u:Uniprot {uniprot_id: $uniprot_id})
MERGE (up:UniprotProperty {type: $type, property: $property})
MERGE (u)-[:HAS_UNIPROT_PROPERTY]->(up)
'''

# def populate_query(d):
#     kv_str = []
#     for k, v in d.items():
#         if not v or k=="From": continue
#         # kv_str.append(create_uniprot_nodes.replace("KV", f"type: \'{k}\', property: \'{v}\'"))
#         kv_str.append(create_uniprot_nodes.replace("KV", f"type: \'{k}\', property: \'{v}\'"))

#     return kv_str

# res = populate_query(uniprot[0])

count = 0
total = len(uniprot)

def add_uniprot_triple(i, d):
    global count, total
    if not count&127 and count!=0: print(f"processed {count}/{total} data slices")
    graph.run(create_uniprot_index, protein_id=d["From"], uniprot_id=d["Entry"])
    for k,v in d.items():
        if not v or k=="From": continue
        graph.run(create_uniprot_nodes, uniprot_id=d["Entry"], type=k, property=v)
    count+=1

In [ ]:
print('Creating pool')
with Pool(100) as p:
  print('Running map...')
  p.starmap(add_uniprot_triple, enumerate(uniprot))
  print('Queries complete. Destroying pool')
print('Pool complete.')

# Search Tree

## Retrieving the Genes and their Node ids

In [9]:
id_map = graph.run('MATCH (g:Sequence) RETURN {g: g, id: elementId(g)}')

In [11]:
id_map = id_map.data()

In [12]:
len(id_map)
id_map[:10]

[{'{g: g, id: elementId(g)}': {'id': '4:c81c1c99-e897-44b9-b8cc-e4b6e264a420:0',
   'g': Node('Sequence', exon_id='ENSE00001943628', level=12, limit=1.2786107063293457, protein_id='ENSP00000466958', sequence='TACAGGCAGAAAACTACCTGGCACTGTGGGAGACTGAGTATAGGAGAAAGCAGGATGACGATGGGCCTGAATGGTGGTGTGTTTTCCTCCACACAATGTGGCTTTGTCCTCCAAGGTGTCTCACGTGATGGAAGCTGGCTGATCTGACTTGCTTG', thresh=1.2786107063293457)}},
 {'{g: g, id: elementId(g)}': {'id': '4:c81c1c99-e897-44b9-b8cc-e4b6e264a420:1',
   'g': Node('Sequence', exon_id='ENSE00003602238', level=11, limit=1.5374518632888794, protein_id='ENSP00000317992', sequence='CTGGTGTCCTGTCTGTCGGAGACGACGGTGTTGGCGGCCGTGCTGCGGCACATCAGCGTGCTGGTGCCCTGCTTCCTGACCTTCCCCAAGCAGTGCCGCATGCTGCTCAAG', thresh=1.465311884880066)}},
 {'{g: g, id: elementId(g)}': {'id': '4:c81c1c99-e897-44b9-b8cc-e4b6e264a420:2',
   'g': Node('Sequence', exon_id='ENSE00001688790', level=11, limit=1.5303996801376343, protein_id='ENSP00000368678', sequence='GCCCCACCTTCGCCCCGCTGCCGCCTGTGGCCCCCTTACACT

In [13]:
id_pairs: list[tuple[str, str]] = [(v['id'], v['g']['sequence']) for r in id_map for v in [r.values().__iter__().__next__()]]

print(len(id_pairs), 'pairs')

id_pairs[:5]

19995 pairs


[('4:c81c1c99-e897-44b9-b8cc-e4b6e264a420:0',
  'TACAGGCAGAAAACTACCTGGCACTGTGGGAGACTGAGTATAGGAGAAAGCAGGATGACGATGGGCCTGAATGGTGGTGTGTTTTCCTCCACACAATGTGGCTTTGTCCTCCAAGGTGTCTCACGTGATGGAAGCTGGCTGATCTGACTTGCTTG'),
 ('4:c81c1c99-e897-44b9-b8cc-e4b6e264a420:1',
  'CTGGTGTCCTGTCTGTCGGAGACGACGGTGTTGGCGGCCGTGCTGCGGCACATCAGCGTGCTGGTGCCCTGCTTCCTGACCTTCCCCAAGCAGTGCCGCATGCTGCTCAAG'),
 ('4:c81c1c99-e897-44b9-b8cc-e4b6e264a420:2',
  'GCCCCACCTTCGCCCCGCTGCCGCCTGTGGCCCCCTTACACTGTGCCCAGACGCCCTACGGCTGCTGCCAGGACAATATCACCGCAGCCCGGGGCGTGGGCCTGGCTGGCTGCCCCA'),
 ('4:c81c1c99-e897-44b9-b8cc-e4b6e264a420:3',
  'ATGCTCTACCTGCTGTGCTCCTGGCCGGAGCTGCCCGTCCTGAGCGCCCTGGAGCTGCTAGACTTCAGCTTCCCCGATTGCCACGTAGGCTCCTTCGCCATCAAGTCGCTGCGGAAACTGAC'),
 ('4:c81c1c99-e897-44b9-b8cc-e4b6e264a420:4',
  'ATCGGCTCCGTGGAGCGCTTCATCCACCACGTGAACGCGTGCATGCGGCAGCGGCAGGAGCGGCAGCGGCTGGCGGCCGTGGTGAGCCGCATCGACGCCTACGAGGTGGTGGAAAGCAGCAGCGACGAAGTGGACAAG')]

In [14]:
import os

os.environ["WANDB_DISABLED"] = "true"
model_name = "zhihan1996/DNABERT-2-117M"
path = "./data/dnabert2_model"
tokenizer, model = None, None
tokenizer_path, model_path = path+"/tokenizer/"+model_name, path+"/model/"+model_name

if os.path.isdir(tokenizer_path) and os.path.isdir(model_path):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_path, trust_remote_code=True)
    model = AutoModel.from_pretrained(model_path, trust_remote_code=True)
else:
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    model = AutoModel.from_pretrained(model_name, trust_remote_code=True)
    tokenizer.save_pretrained(tokenizer_path)
    model.save_pretrained(model_path)

# cuda_model = model.to("cuda")

/home/daftylooper/Desktop/Capstone/code/finetune-chatgpt-go/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/daftylooper/Desktop/Capstone/code/finetune-chatgpt-go/.venv/lib/python3.10/site-packages/transformers/modeling_utils.py:446: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longe

## Distance function using DNABERT-2

In [15]:
def dnabert_embedding_mean(dna):
    # inputs = tokenizer(dna, return_tensors = 'pt')["input_ids"].to('cuda')
    inputs = tokenizer(dna, return_tensors = 'pt')["input_ids"]
    hidden_states = model(inputs)[0] # [1, sequence_length, 768]
    # print(inputs.shape)
    # print(hidden_states.shape)

    # embedding with mean pooling
    embedding_mean = torch.mean(hidden_states[0], dim=0)
    # print(embedding_mean.shape) # expect to be 768
    # print(embedding_mean)
    
    # # embedding with max pooling
    # embedding_max = torch.max(hidden_states[0], dim=0)[0]
    # print(embedding_max.shape) # expect to be 768
    
    return embedding_mean.detach()

In [16]:
def dnabert_embedding_distance(a, b):

    try:
        vd1 = memo[a]
    except KeyError:
        print("KeyError: a:", a)
        
    try:
        vd2 = memo[b]
    except KeyError:
        print("KeyError: b:", b)
    
    return float(torch.linalg.vector_norm(vd1-vd2))

## Calculate and memoize embeddings. Store for future sessions.

In [ ]:
import numpy as np

def embedding_maker_worker(i, tup):
    id, seq = tup
    with torch.no_grad():
        if not i&127: print(f"{i}: {len(seq)}")
        arr.append(len(seq))
        memo[seq] = dnabert_embedding_mean(seq)

# print(f"first seven - {arr[:8]}")
# print(f"avg seq len {np.mean(arr)}")

In [17]:
import os
import ast
import csv

path = "./data/working/dnabert2_embeddings.csv"
memo = {}
arr = []

if os.path.exists(path):
    print("Found existing dump of embeddings, loading them...")
    with open(path, "r") as f:
        reader = csv.reader(f)
        for row in reader:
            seq, embedding = row
            embedding = torch.tensor(ast.literal_eval(embedding))
            memo[seq] = embedding
else:
    print("Embeddings don't exist, creating...")
    with Pool(200) as p:
      print('Running map...')
      p.starmap(embedding_maker_worker, enumerate(id_pairs[::-1]))
      print('Queries complete. Destroying pool')

    print("Writing Rows...")
    with open(path, "w") as f:
        writer = csv.writer(f)
        for seq, embd in memo.items():
            writer.writerow([seq, embd.tolist()])

print("Done!")

Found existing dump of embeddings, loading them...
Done!


In [ ]:
# !rm "./data/working/dnabert2_embeddings.csv"

In [18]:
memo["CCCAGATCTCTTCAG"]
print(f"size of id_pairs: {len(id_pairs)}")
print(f"size of memo: {len(memo)}")

size of id_pairs: 19995
size of memo: 8472


In [19]:
print(dnabert_embedding_distance(
  "CCCAGATCTCTTCAG",
  "TTTTTATGCCTCATTCTGTGAAAATTGCTGTAGTCTCTTCCAGTTATGAAGAAG",
))

print(len(memo))

6.9764404296875
8472


## Construction

In [ ]:
query = '''

  MATCH (r:Sequence WHERE elementId(r) = $root_id)

  MATCH (n:Sequence WHERE elementId(n) = $n_id)

  SET r.thresh = $thresh

  SET r.limit = $limit

  SET n.level = $level

  CREATE (r)-[:SEARCH_BRANCH {branch_type: $branch_type}]->(n)

'''

In [ ]:
def create_tree_instructions(graph, id_pairs: list[tuple[str, str]], distance: Callable[[Sequence, Sequence], float], r: Random, level = 0) -> tuple[str, str, list[dict]] | None:

  if not id_pairs: return None

  out = []



  l = len(id_pairs)

  root_idx = r.randrange(l)

  root_id, root = id_pairs.pop(root_idx)



  s = sorted(id_pairs, key=lambda g: distance(root, g[1]))

  split_pos = l//2

  if not s: limit = thresh = 0.0

  elif l//2 >= len(s): limit = thresh = distance(root, s[-1][1])

  else:
    print(s)

    distances = [distance(root, g[1]) for g in s]

    thresh = distances[l//2]

    limit = distances[-1]

    # if limit != thresh: print('EXPECTED', thresh, limit)

    split_pos = bisect(distances, thresh)

    # out.append({'id': root_id, 'limit': limit, 'thresh': thresh, 'far_subtree': (distances[split_pos], distances[-1])})



  near_result = create_tree_instructions(graph, s[:split_pos], distance, r, level=level+1)

  if near_result is not None:

    out.extend(near_result[2])

    out.append({

        "branch_type": "NEAR",

        "root_id": root_id,

        "n_id": near_result[0],

        "thresh": thresh,

        "level": level+1,

        "limit": limit,

    })



  far_result = create_tree_instructions(graph, s[split_pos:], distance, r, level=level+1)

  if far_result is not None:

    out.extend(far_result[2])

    out.append({

        "branch_type": "FAR",

        "root_id": root_id,

        "n_id": far_result[0],

        "thresh": thresh,

        "level": level+1,

        "limit": limit,

    })



  print(level, root_id, root[:64])



  return root_id, root, out

In [ ]:
def edit_distance(a: Sequence, b: Sequence) -> float:  # levenshtein distance

  ai = 0

  bi = 0

  d = 0.0

  while ai < len(a) and bi < len(b):

    if a[ai] == b[bi]:  # no edit

      ai += 1

      bi += 1

    elif ai > 0 and a[ai-1] == b[bi]:  # add to a / delete from b

      d += 1

      bi += 1

    elif bi > 0 and a[ai] == b[bi-1]:  # add to b / delete from a

      d += 1

      ai += 1

    else:  # replacement

      d += 1

      ai += 1

      bi += 1



  d += len(a)-ai

  d += len(b)-bi



  return d

In [ ]:
# seed = randrange(1<<32)

# seed = 0x509c2232

seed = 0xf67832cb

print(f'seed = 0x{seed:08x}')

r = Random(seed)
     
# root_id, root, kwarg_list = create_tree_instructions(graph, id_pairs.copy(), edit_distance, r)
root_id, root, kwarg_list = create_tree_instructions(graph, id_pairs.copy(), dnabert_embedding_distance, r)

root_id, root

In [ ]:
# Takes 1-2 seconds. *Blazingly Fast*

print('Creating pool')

with Pool(100) as p:

  print('Running map...')

  p.map(

    lambda kwargs: graph.run(query, **kwargs),

    kwarg_list,

  )

  print('Queries complete. Destroying pool')

print('Pool complete.')

## Verification

In [ ]:
def tree_testing(graph, id_pairs: list[tuple[str, str]], distance: Callable[[Sequence, Sequence], float], r: Random, level = 0) -> tuple[str | None, str | None, list[dict]]:

  if not id_pairs: return None, None, []

  out = []



  l = len(id_pairs)

  root_idx = r.randrange(l)

  root_id, root = id_pairs.pop(root_idx)



  s = sorted(id_pairs, key=lambda g: distance(root, g[1]))

  split_pos = l//2

  if not s:

    thresh = 0.0

    limit = 0.0

  elif l//2 >= len(s): limit = thresh = distance(root, s[-1])

  else:

    distances = [distance(root, g[1]) for g in s]

    thresh = distances[l//2]

    limit = distances[-1]

    if limit != thresh: print('EXPECTED', thresh, limit)

    split_pos = bisect(distances, thresh)

    out.append({'id': root_id, 'limit': limit, 'thresh': thresh, 'far_subtree': (distances[split_pos], distances[-1])})



  near_result = tree_testing(graph, s[:split_pos], distance, r, level=level+1)

  out.extend(near_result[2])

  far_result = tree_testing(graph, s[split_pos:], distance, r, level=level+1)

  out.extend(far_result[2])



  if level < 7: print(level, root_id, root[:64])



  return root_id, root, out



seed = 0xf67832cb

print(f'seed = 0x{seed:08x}')

r = Random(seed)

# root_id, root, lev_properties = tree_testing(graph, id_pairs.copy(), lev.distance, r)

root_id, root, lev_properties = tree_testing(graph, id_pairs.copy(), edit_distance, r)

root_id, root

In [ ]:
[prop for prop in lev_properties if len({*prop['far_subtree']}) == 2]

In [ ]:
len(lev_properties)

In [ ]:
root_id

In [ ]:
len(kwarg_list)

In [ ]:
# Check if update was successful

l = [(d['r']['thresh'], d['r']['limit']) for d in graph.run('MATCH  (r:Sequence)-[:SEARCH_BRANCH]->(n) RETURN r, n').data() if d['r']['thresh'] != d['r']['limit']]

print(len(l))

l

In [ ]:
unique_kwargs = {(kwarg['n_id'], kwarg['root_id']) for kwarg in kwarg_list}

len(unique_kwargs)

In [ ]:
for i, kwargs in enumerate(kwarg_list[:15]):

  print('''

    MATCH (r:Gene WHERE elementId(r) = {root_id!r})

    MATCH (n:Gene WHERE elementId(n) = {n_id!r})

    SET r.thresh = {thresh!r}

    CREATE (r)-[:SEARCH_BRANCH {{branch_type: {branch_type!r}}}]->(n)

    RETURN 1

  '''.format(**kwargs))

## Search

## Import Search Tree to Memory

In [20]:
tree_data = graph.run(
  '''
  MATCH (r)-[sb:SEARCH_BRANCH]->(c)
  RETURN {
    name: r.sequence,
    id: elementId(r),
    thresh: r.thresh,
    limit: r.limit,
    child_id: elementId(c),
    branch: sb.branch_type
  }
  '''
).data()

class SearchNode:
	def __init__(self, id, name, thresh, limit):
		self.id = id
		self.name = name
		self.thresh = thresh
		self.limit = limit

		self.nchild_id = None
		self.fchild_id = None

len(tree_data)

branches = [branch.values().__iter__().__next__() for branch in tree_data]
branches[0]

nodes = {}
for branch in branches:
  if branch['id'] in nodes:
    node = nodes[branch['id']]
  else:
    node = nodes[branch['id']] = SearchNode(
        id=branch['id'],
        name=branch['name'],
        thresh=branch['thresh'],
        limit=branch['limit'],
    )

  if branch['branch'] == 'NEAR':
    node.nchild_id = branch['child_id']
  elif branch['branch'] == 'FAR':
    node.fchild_id = branch['child_id']

In [21]:
def search_im(root_id, q, qthresh, paths=1, distance=dnabert_embedding_distance) -> tuple[list[tuple[float, str, str]], int]:
  if root_id not in nodes: return [], paths
  
  root = nodes[root_id]
  d = distance(q, root.name)
  
  if d <= qthresh:
    out = [(d, root_id, root.name)]
  else:
    out = []

  if root.thresh is None: return out, paths

  if abs(d-root.thresh) <= qthresh:
    paths += 1
    # print(f'Paths: {paths}, Diff: {abs(d-root.thresh)}')

  if d-qthresh < root.thresh and root.nchild_id is not None:
    terms, paths = search_im(root.nchild_id, q, qthresh, paths)
    out.extend(terms)
  if d-qthresh < root.thresh and root.fchild_id is not None:
    terms, paths = search_im(root.fchild_id, q, qthresh, paths)
    out.extend(terms)
  
  return out, paths

In [22]:
root_result = graph.run(
  '''
  MATCH (r)-[sb:SEARCH_BRANCH]->(c {level: 1})
  RETURN {id: elementId(r), r: r}
  '''
).data()
root_result

root_id = '4:c81c1c99-e897-44b9-b8cc-e4b6e264a420:1956'
root = nodes[root_id]

def similarity_search_im(q):
    global memo
    t_memo = memo.copy()
    memo[q] = dnabert_embedding_mean(q)
    q_result, paths = search_im(root_id, q, 1)
    memo = t_memo
    return q_result

In [23]:
# q = "GCCCCACGTGTGTGCTGAGCAGGAGCTGACCCTGGTGGGCCGCCGCCAGCCGTGCGTGCAGGCCTTAAGCCACACGGTGCCGGTGTGGAAGGCCGGCTGTGGGTGGCAGGCGTGGTGCGTGGGTCATGAGCGGAG"
q = "TACAGGCAGAAAACTACCTGGCACTGTGGGAGACTGAGTATAGGAGAAAGCAGGATGACGATGGGCCTGAATGGTGGTGTGTTTTCCTCCACACAATGTGGCTTTGTCCTCCAAGGTGTCTCACGTGATGGAAGCTGGCTGATCTGACTTGCTTG"
similarity_search_im(q)
# root

[(0.0,
  '4:c81c1c99-e897-44b9-b8cc-e4b6e264a420:0',
  'TACAGGCAGAAAACTACCTGGCACTGTGGGAGACTGAGTATAGGAGAAAGCAGGATGACGATGGGCCTGAATGGTGGTGTGTTTTCCTCCACACAATGTGGCTTTGTCCTCCAAGGTGTCTCACGTGATGGAAGCTGGCTGATCTGACTTGCTTG')]

In [26]:
def search(root_id, root_name, tree_thresh, tree_limit, q, qthresh, paths=1, distance=dnabert_embedding_distance) -> tuple[list[tuple[float, str, str]], int]:

  # print("dist->")
  # print(q)
  # print(root_name)
  d = distance(q, root_name)
  # print(f"Distance: {d}, {tree_thresh=}")

  if tree_limit is None:
      if d<=qthresh:
        print(f"logging: limitless! {[(d, root_id, root_name)]}, {paths}")
        return [(d, root_id, root_name)], paths
      else:
        return [], paths
      
  if d > qthresh+tree_limit:
    return [], paths
  
  if tree_limit <= qthresh-d:

    print('SUBTREE BATCH')

    results = graph.run(

      '''

      MATCH (r WHERE elementId(r) = $root_id)-[:SEARCH_BRANCH*1..]->(c)

      RETURN {child_id: elementId(c), child: c.sequence}

      ''',

      root_id=root_id,

    ).data()



    out = []

    for result in results:

      [values] = [*result.values()]

      out.append((distance(q, values['child']), values['child_id'], values['child']))

    if out: print(f"logging: found a substree: {out}")
    return out, paths



  elif d <= qthresh:

    out = [(d, root_id, root_name)]
    print(f"logging, d is less than qthresh: {out}")

  else:

    out = []



  if tree_thresh is None: return out, paths



  result = graph.run(

    '''

    MATCH (r WHERE elementId(r) = $root_id)-[sb:SEARCH_BRANCH]->(c)

    RETURN {child_id: elementId(c), child: c.sequence, thresh: c.thresh, limit: c.limit, branch: sb.branch_type, level: c.level}

    ''',

    root_id=root_id,

  ).data()

  level = None
  children: dict[str, tuple[str, str, str, str]] = {}

  for child in result:
  
    [values] = [*child.values()]
    level = values.get("level", None)
    # print(values)
    children[values['branch']] = values['child_id'], values['child'], values['thresh'], values["limit"]



  if abs(d-tree_thresh) <= qthresh:

    paths += 1

    print(f'Paths: {paths}, Diff: {d-tree_thresh}, Level: {level}')


  # print("near:", children["NEAR"])
  # print("far:", children["FAR"])
    
  if d-qthresh < tree_thresh and 'NEAR' in children:
    # print("NEAR:", children['NEAR'])
    terms, paths = search(*children['NEAR'], q, qthresh, paths)

    out.extend(terms)

  if d+qthresh >= tree_thresh and 'FAR' in children:
    # print("FAR:", children['FAR'])
    terms, paths = search(*children['FAR'], q, qthresh, paths)

    out.extend(terms)



  return (out, paths)

## Searching on exon graph

In [24]:
root_thresh = 1.7634133100509644  # queried from the neo4j console for ({level: 0})
root_limit = 6.264886856079102  # queried from the neo4j console for ({level: 0})
root_id, root = ('4:c81c1c99-e897-44b9-b8cc-e4b6e264a420:1956', 'GCCCCACGTGTGTGCTGAGCAGGAGCTGACCCTGGTGGGCCGCCGCCAGCCGTGCGTGCAGGCCTTAAGCCACACGGTGCCGGTGTGGAAGGCCGGCTGTGGGTGGCAGGCGTGGTGCGTGGGTCATGAGCGGAG')

In [25]:
q_result, paths = search(root_id, root, root_thresh, root_limit, "CCGATTCCACGACTCTTCTTTGCGGGAGAACATACGATCCGTAACTACCCAGCCACAGTGCATGGTGCTCTGCTGAGTGGGCTGCGAGAAGCGGGAAGAATTGCAGACCAGTTTTTGGGGGCCATGTATACGCTGCCTCGCCAGGCCACACCAGGTGTTCCTGCACAGCAGTCCCCAAGCATGTGAGACAGATGCATTCTAAGGGAAGAGGCCCATGTGCCTGTTTCTGCCATGTAAGGAAGGCTCTTCTAGCAATACTAGATCCCACTGAGAAAATCCACCCTGGCATCTGGGCTCCTGATCAGCTGATGGAGCTCCTGATTTGACAAAGGAGCTTGCCTCCTTTGAATGACCTAGAGCACAGGGAGGAACTTGTCCATTAGTTTGGAATTGTGTTCTTCGTAAAGACTGAGGCAAGCAAGTGCTGTGAAATAACATCATCTTAGTCCCTTGGTGTGTG", 0.1)
print(len(q_result), 'similar genes')
print("results!\n-----------")
print(q_result)
print(paths)

NameError: name 'search' is not defined

In [ ]:
# change after review
def similarity_search(q):
    global memo
    t_memo = memo.copy()
    memo[q] = dnabert_embedding_mean(q)
    q_result, paths = search(root_id, root, root_thresh, root_limit, q, 0.3)
    memo = t_memo
    return q_result

In [ ]:
q = "CCCAGATCTCTTCAG"
# print(memo[q])
similarity_search("CCGATTCCACGACTCTTCTTTGCGGGAGAACATACGATCCGTAACTACCCAGCCACAGTGCATGGTGCTCTGCTGAGTGGGCTGCGAGAAGCGGGAAGAATTGCAGACCAGTTTTTGGGGGCCATGTATACGCTGCCTCGCCAGGCCACACCAGGTGTTCCTGCACAGCAGTCCCCAAGCATGTGAGACAGATGCATTCTAAGGGAAGAGGCCCATGTGCCTGTTTCTGCCATGTAAGGAAGGCTCTTCTAGCAATACTAGATCCCACTGAGAAAATCCACCCTGGCATCTGGGCTCCTGATCAGCTGATGGAGCTCCTGATTTGACAAAGGAGCTTGCCTCCTTTGAATGACCTAGAGCACAGGGAGGAACTTGTCCATTAGTTTGGAATTGTGTTCTTCGTAAAGACTGAGGCAAGCAAGTGCTGTGAAATAACATCATCTTAGTCCCTTG")

here here

In [ ]:
q = "CCCAGATCTCTTCAG"
# print(memo[q])
similarity_search("CCGATTCCACGACTCTTCTTTGCGGGAGAACATACGATCCGTAACTACCCAGCCACAGTGCATGGTGCTCTGCTGAGTGGGCTGCGAGAAGCGGGAAGAATTGCAGACCAGTTTTTGGGGGCCATGTATACGCTGCCTCGCCAGGCCACACCAGGTGTTCCTGCACAGCAGTCCCCAAGCATGTGAGACAGATGCATTCTAAGGGAAGAGGCCCATGTGCCTGTTTCTGCCATGTAAGGAAGGCTCTTCTAGCAATACTAGATCCCACTGAGAAAATCCACCCTGGCATCTGGGCTCCTGATCAGCTGATGGAGCTCCTGATTTGACAAAGGAGCTTGCCTCCTTTGAATGACCTAGAGCACAGGGAGGAACTTGTCCATTAGTTTGGAATTGTGTTCTTCGTAAAGACTGAGGCAAGCAAGTGCTGTGAAATAACATCATCTTAGTCCCTTG")

In [ ]:
similarity_search("GTCTGTGATGAGGAATGGCACCACTACGTCCTCAATGTAGAATTCCCGAGTGTGACTCTCTATGTGGATGGCACGTCCCACGAGCCCTTCTCTGTGACTGAGGATTACCCGCTCCATCCATCCCTCAGCTCGTGGTGGGGGCTTGCTGGCAAG")

In [ ]:
similarity_search(root)

In [ ]:
print(similarity_search(root))

seqs = ["GGGCCCCCTCATAAATGTGCCTTAATTTTCGCAGATAACAGTGGAATAGACATCATTTTGGGAGTCTTCCCCTTTGTCAGGGAGCTACTCCTTAGAGGGACAGAG",
        "AGCCTGAAGCCGCTGCTGGAGAAGCGCCGGCGCGCGCGCATCAACCAGAGCCTGAGCCAGCTTAAGGGGCTCATCCTGCCGCTGCTGGGCCGGGAG",
        "GTCTGTGATGAGGAATGGCACCACTACGTCCTCAATGTAGAATTCCCGAGTGTGACTCTCTATGTGGATGGCACGTCCCACGAGCCCTTCTCTGTGACTGAGGATTACCCGCTCCATCCATCCAAGATAGAAACTCAGCTCGTGGTGGGGGCTTGCTGGCAAG",
        "GCCCAGTTCAATCTGCTGAGCAGCACCATGGACCAGATGAGCAGCCGCGCGGCCTCGGCCAGCCCCTACACCCCAGAGCACGCCGCCAGCGTGCCCACCCACTCGCCCTACGCACAACCCAGCTCCACCTTCGACACCATGTCGCCGGCGCCTGTCATCCCCTCCAACACCGACTACCCCGGACCCCACCACTTTGAGGTCACTTTCCAGCAGTCCAGCACGGCCAAGTCAGCCACCTGGACG",
        "GCTCGCGGGACCCCTGCTCCAACGTGACCTGCAGCTTCGGCAGCACCTGTGCGCGCTCGGCCGACGGGCTGACGGCCTCGTGCCTGTGCCCCGCGACCTGCCGTGGCGCCCCCGAGGGGACCGTCTGCGGCAGCGACGGCGCCGACTACCCCGGCGAGTGCCAGCTCCTGCGCCGCGCCTGCGCCCGCCAGGAGAATGTCTTCAAGAAGTTCGACGGCCCTTGTG"]

out = []
for x in seqs:
    out.append(similarity_search(x))

for res in out:
    print(res)

In [27]:
traverse_similar_node = '''
MATCH (s:Sequence WHERE elementId(s) = $node_id)
MATCH (s)-[:MAPS_TO]->(t:Transcript)
MATCH (t)-[:PART_OF]->(g:Gene)
MATCH (g)-[:HAS_GO]->(go:Go)
MATCH (go)-[ra]->(gop:GoProperty)
MATCH (s)-[:HAS_UNIPROT]->(u:Uniprot)
MATCH (u)-[rb]->(up:UniprotProperty)
return s, t, g, go, gop, u, up;
'''

def get_prop(node_id):
    res = graph.run(traverse_similar_node, node_id=node_id)
    return res.data()

# remove all redundant data
# note: exons are same??
# note: in go property nodes, include property type. fetching the type from relation is SLOW.
def parse_prop_output(data):
    out = {
    "sequence": set(),
    "exon_id": set(),
    "protein_id": set(),
    "transcript_id": set(),
    "hgnc_id": set(),
    "go_id": set(),
    "gop_property": set(),
    "uniprot_id": set(),
    "up_property": set()
    }

    for slice in data:
        for kv in slice.keys():
            if kv=="up" or kv=="gop": # is a go proety or uniprot property, append different dta slice
                out[str(kv)+"_property"].add((slice[kv]["property"], slice[kv]["type"])) # applied to uniprot or go properties
            else:
                for k,v in slice[kv].items():
                    if k=="property":
                        out[str(kv)+"_"+str(k)].add(v)
                    if k in out:
                        out[k].add(v)

    for k,v in out.items():
        out[k] = list(out[k])

    return out

In [ ]:
import json

json.loads(json.dumps(get_prop(root_id)))

In [28]:
import json

def process_dict(d):
    gop = []
    for slice in d["gop_property"]:
        gop.append({slice[0]: slice[1]})
    up = []
    for slice in d["up_property"]:
        up.append({slice[0]: slice[1]})
    
    d["gop_property"] = gop
    d["up_property"] = up

    return d

# q = "CCCAGATCTCTTCAG"
def query_sequence(q):
    # res = similarity_search(q)
    res = similarity_search_im(q)
    print(f"result! {res}")
    res_id = None
    if res:
        res_id = res[0][1]
    res = get_prop(res_id)
    res = json.loads(json.dumps(res))
    res = parse_prop_output(res)
    res = process_dict(res)

    return res

In [29]:
res = query_sequence("TACAGGCAGAAAACTACCTGGCACTGTGGGAGACTGAGTATAGGAGAAAGCAGGATGACGATGGGCCTGAATGGTGGTGTGTTTTCCTCCACACAATGTGGCTTTGTCCTCCAAGGTGTCTCACGTGATGGAAGCTGGCTGATCTGACTTGCTTG")
res

result! [(0.0, '4:c81c1c99-e897-44b9-b8cc-e4b6e264a420:0', 'TACAGGCAGAAAACTACCTGGCACTGTGGGAGACTGAGTATAGGAGAAAGCAGGATGACGATGGGCCTGAATGGTGGTGTGTTTTCCTCCACACAATGTGGCTTTGTCCTCCAAGGTGTCTCACGTGATGGAAGCTGGCTGATCTGACTTGCTTG')]


{'sequence': ['TACAGGCAGAAAACTACCTGGCACTGTGGGAGACTGAGTATAGGAGAAAGCAGGATGACGATGGGCCTGAATGGTGGTGTGTTTTCCTCCACACAATGTGGCTTTGTCCTCCAAGGTGTCTCACGTGATGGAAGCTGGCTGATCTGACTTGCTTG'],
 'exon_id': ['ENSE00001943628'],
 'protein_id': ['ENSP00000466958'],
 'transcript_id': ['ENST00000474874'],
 'hgnc_id': ['HGNC:18185'],
 'go_id': ['GO:0045616',
  'GO:0060428',
  'GO:0005829',
  'GO:0032570',
  'GO:0060613',
  'GO:0050847',
  'GO:0019901',
  'GO:0036022',
  'GO:0009410',
  'GO:0005886',
  'GO:0007173',
  'GO:0001889',
  'GO:0016477',
  'GO:0043588',
  'GO:0060065',
  'GO:0042632',
  'GO:0006006',
  'GO:0007566',
  'GO:0048545',
  'GO:1904565',
  'GO:0035847',
  'GO:0031953',
  'GO:0005737',
  'GO:0006699'],
 'gop_property': [{'GO:0005829': 'has_id'},
  {'GO:0007566': 'has_id'},
  {'GO:0005737': 'has_id'},
  {'GO:0060612': 'is_a'},
  {'GO:0019901': 'has_id'},
  {'cytoplasm': 'has_name'},
  {'cell migration': 'has_name'},
  {'GO:0031952': 'is_a'},
  {'GO:0009653': 'is_a'},
  {'progesterone receptor s

# Benchmark

In [167]:
import random

s, f = [0]*1000, [0]*1000
t_memo = [*memo]

for edits in range(10,1000,10):
    for i, q in enumerate(t_memo):
        if not i&255: print(f"processed {i} sequences for {edits} edits!")
        res = similarity_search_im(q[:-edits])
        if res: s[edits] += 1
        else: f[edits] += 1

print(f"success {s}")
print(f"failure {f}")

processed 0 sequences for 10 edits!


processed 256 sequences for 10 edits!


/home/daftylooper/.cache/huggingface/modules/transformers_modules/zhihan1996/DNABERT-2-117M/d064dece8a8b41d9fb8729fbe3435278786931f1/bert_layers.py:433: UserWarning: Increasing alibi size from 2158 to 3171
  warnings.warn(


processed 512 sequences for 10 edits!
processed 768 sequences for 10 edits!
processed 1024 sequences for 10 edits!
processed 1280 sequences for 10 edits!
processed 1536 sequences for 10 edits!
processed 1792 sequences for 10 edits!
processed 2048 sequences for 10 edits!
processed 2304 sequences for 10 edits!
processed 2560 sequences for 10 edits!
processed 2816 sequences for 10 edits!
processed 3072 sequences for 10 edits!
processed 3328 sequences for 10 edits!
processed 3584 sequences for 10 edits!
processed 3840 sequences for 10 edits!
processed 4096 sequences for 10 edits!
processed 4352 sequences for 10 edits!
processed 4608 sequences for 10 edits!
processed 4864 sequences for 10 edits!
processed 5120 sequences for 10 edits!
processed 5376 sequences for 10 edits!
processed 5632 sequences for 10 edits!
processed 5888 sequences for 10 edits!
processed 6144 sequences for 10 edits!
processed 6400 sequences for 10 edits!
processed 6656 sequences for 10 edits!
processed 6912 sequences fo

KeyboardInterrupt: 

In [170]:
s[10], f[10]

(6069, 2404)

In [1]:
s[10]/(s[10]+f[10])

NameError: name 's' is not defined

In [171]:
f1, f2 = open("success.txt", "w"), open("failure.txt", "w")
print(s, file=f1)
print(f, file=f2)

# Natural Language Model

## Instantiate Client

In [30]:
import os
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY")
)

# gpt-3.5-turbo-0125

model_params = {
    "model": "gpt-4-0125-preview",
    "max_tokens": 1024,
    "temperature": 0.7,
    "top_p": 0.9,
}

def chat_completion(instruction, query):
    return client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": instruction
            },
            {
                "role": "user",
                "content": query
            }
        ],
        **model_params
    )

## Call Model

In [37]:
import json

system_instruction = '''
You are a bioinformatics expert providing detailed analysis of structured JSON data derived from biological databases, 
such as Gencode, UniProt, and GO. For each data element, give a thorough description of its biological significance, 
with insights into gene function, protein structure, molecular and cellular roles, and potential interactions or pathways. 
Where applicable, refer to relevant database annotations to enrich the analysis.
'''

user_content = '''
Analyze the structured JSON data provided, which contains detailed information derived from multiple biological databases, 
including Gencode, UniProt, and GO, related to specific gene and protein sequences. For each data element, provide a detailed description 
of its biological significance:\n\n
Sequence: Describe the likely function or properties of the gene sequence based on its structure or known associations.\n
Exon ID: Explain the significance of this exon and its role in gene expression.\n
Protein ID and UniProt properties: Summarize the protein encoded by this gene, detailing its structure (domains, biases) and function, 
referring to UniProt annotations where applicable.\n
Transcript ID: Describe the relevance of this transcript and how it might relate to the gene's overall functionality.\n
HGNC ID: Give an overview of the gene's identity, nomenclature, and recognized functions.\n
GO IDs and Properties: Describe the biological processes, cellular components, or molecular functions associated with the gene/protein, 
using GO ontology terms and definitions.\n\n
Using this information, create a coherent summary highlighting key biological insights, including gene function, protein structure and 
activity, cellular roles, and any inferred interactions or pathways.\n\n

Return the answer in a presentable, formatted way and limit your response to ~800 tokens
'''

def call_natural_language_model(data):
    if not data["exon_id"]: return
    superprompt = user_content+str(data)
    completion = chat_completion(system_instruction, superprompt).to_dict()
    return completion
    

In [38]:
res = query_sequence(q)
completion = call_natural_language_model(res)

result! [(0.0, '4:c81c1c99-e897-44b9-b8cc-e4b6e264a420:0', 'TACAGGCAGAAAACTACCTGGCACTGTGGGAGACTGAGTATAGGAGAAAGCAGGATGACGATGGGCCTGAATGGTGGTGTGTTTTCCTCCACACAATGTGGCTTTGTCCTCCAAGGTGTCTCACGTGATGGAAGCTGGCTGATCTGACTTGCTTG')]


In [39]:
completion

{'id': 'chatcmpl-AT3yGady9XaoQhl19fzC4XObQ6gMK',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'logprobs': None,
   'message': {'content': "Based on the structured JSON data provided, a comprehensive biological analysis of the specified gene and its encoded protein, including their roles, functions, and interactions, can be compiled as follows:\n\n### Gene and Sequence Analysis\nThe provided nucleotide sequence represents a segment of DNA that encodes for a specific protein. The sequence's composition and order are crucial for determining the protein's structure and function. The sequence information alone doesn't reveal the gene's function but serves as a foundational element in gene expression and regulation studies.\n\n### Exon ID: ENSE00001943628\nThis exon ID refers to a specific segment within a gene that is transcribed and retained in the mRNA after splicing. Exons play critical roles in gene expression, as the combination of different exons through alternative splici

# **Flask Server**

In [ ]:
import nest_asyncio
from flask import Flask, request, jsonify
import json
from flask_cors import CORS

nest_asyncio.apply()

app = Flask(__name__)
CORS(app)

@app.route('/')
# maybe list all routes on index, acting as a, well index
def home():
    return "Hello, World!"

@app.route('/sequence', methods=['GET'])
def process_sequence():
    sequence = request.headers.get('sequence')
    if not sequence:
        return jsonify({"error": "Sequence header is missing"}), 400

    result = query_sequence(sequence)
    result = call_natural_language_model(result)
    result_json_str = json.dumps(result)

    return result_json_str, 200, {'Content-Type': 'application/json'}

app.run(port=4000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:4000
Press CTRL+C to quit
127.0.0.1 - - [13/Nov/2024 14:59:46] "OPTIONS /sequence HTTP/1.1" 200 -


result! [(0.0, '4:c81c1c99-e897-44b9-b8cc-e4b6e264a420:0', 'TACAGGCAGAAAACTACCTGGCACTGTGGGAGACTGAGTATAGGAGAAAGCAGGATGACGATGGGCCTGAATGGTGGTGTGTTTTCCTCCACACAATGTGGCTTTGTCCTCCAAGGTGTCTCACGTGATGGAAGCTGGCTGATCTGACTTGCTTG')]


127.0.0.1 - - [13/Nov/2024 15:00:14] "GET /sequence HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2024 15:00:58] "OPTIONS /sequence HTTP/1.1" 200 -


result! [(0.33635762333869934, '4:c81c1c99-e897-44b9-b8cc-e4b6e264a420:0', 'TACAGGCAGAAAACTACCTGGCACTGTGGGAGACTGAGTATAGGAGAAAGCAGGATGACGATGGGCCTGAATGGTGGTGTGTTTTCCTCCACACAATGTGGCTTTGTCCTCCAAGGTGTCTCACGTGATGGAAGCTGGCTGATCTGACTTGCTTG')]


...

127.0.0.1 - - [13/Nov/2024 15:14:34] "GET /sequence HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2024 15:39:48] "OPTIONS /sequence HTTP/1.1" 200 -


result! [(0.8998969197273254, '4:c81c1c99-e897-44b9-b8cc-e4b6e264a420:18606', 'GACCAGTATCAGCTGTGCTACCGTGCGGCCCTGGAGTACCTCGGCAGCTTTGACCACTATGCAACGTAACTACCGCTCCCCTCTCCTCCGCCACCCCCGCCGTGGGGCTCCGGAGGGGACCCAGCTCCTCTGAGCCATACCGACCATCGTCCAGCCCTCCTACGCAGATGCTGTCACTGGCAGAGCACAGCCCACGGGGATCACAGCGTTTCAGGAACGTTGCCACACCAATCAGAGAGCCTAGAACATCCCTGGGCAAGTGGATGGCCCAGCAGGCAGGCACTGTGGCCCTTCTGTCCACCAGACCCACCTGGAGCCCGCTTCAAGCTCTCTGTTGCGCTCCCGCATTTCTCATGCTTCTTCTCATGGGGTGGGGTTGGGGCAAAGCCTCCTTTTTAATACATTAAGTGGGGTAGACTGAGG'), (0.9556918740272522, '4:c81c1c99-e897-44b9-b8cc-e4b6e264a420:12942', 'GACTATGAGCTCTTCGTGGAGGCTGTGGAACAAAACACGCTGCAGGAGTTCCTGAAGCTGGCTTGAGTCAAGCCTGTCCAGAGTTCCCCTGCTGGACTCCATCACCACACTCCCCCCAGCCTTCACCTGGCCATGAAGGACCTTTTGACCAACTCCCTGTCATTCCTAACCTAACCTTAGAGTCCCTCCCCCAATGCAGGCCACTTCTCCTCCCTCCTCTCTAAATGTAGTCCCCTCTCCTCCATCTAAAGGCAACATTCCTTACCCATTAGTCTCAGAAATTGTCTTAAGCAACAGCCCCAAATGCTGGCTGCCCCCAGCCAAGCATTGGGGCCGCCATCCTGCCTGGCACTGGCTGATGGGCACCTCTGTTGGTTCCATCAGCCAGAGCTCTGCCAAAGGCCCCGCAGTCCCTCTCCCAGGA

127.0.0.1 - - [13/Nov/2024 15:40:13] "GET /sequence HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2024 15:40:38] "OPTIONS /sequence HTTP/1.1" 200 -


result! [(0.9429029822349548, '4:c81c1c99-e897-44b9-b8cc-e4b6e264a420:18606', 'GACCAGTATCAGCTGTGCTACCGTGCGGCCCTGGAGTACCTCGGCAGCTTTGACCACTATGCAACGTAACTACCGCTCCCCTCTCCTCCGCCACCCCCGCCGTGGGGCTCCGGAGGGGACCCAGCTCCTCTGAGCCATACCGACCATCGTCCAGCCCTCCTACGCAGATGCTGTCACTGGCAGAGCACAGCCCACGGGGATCACAGCGTTTCAGGAACGTTGCCACACCAATCAGAGAGCCTAGAACATCCCTGGGCAAGTGGATGGCCCAGCAGGCAGGCACTGTGGCCCTTCTGTCCACCAGACCCACCTGGAGCCCGCTTCAAGCTCTCTGTTGCGCTCCCGCATTTCTCATGCTTCTTCTCATGGGGTGGGGTTGGGGCAAAGCCTCCTTTTTAATACATTAAGTGGGGTAGACTGAGG'), (0.986037015914917, '4:c81c1c99-e897-44b9-b8cc-e4b6e264a420:12942', 'GACTATGAGCTCTTCGTGGAGGCTGTGGAACAAAACACGCTGCAGGAGTTCCTGAAGCTGGCTTGAGTCAAGCCTGTCCAGAGTTCCCCTGCTGGACTCCATCACCACACTCCCCCCAGCCTTCACCTGGCCATGAAGGACCTTTTGACCAACTCCCTGTCATTCCTAACCTAACCTTAGAGTCCCTCCCCCAATGCAGGCCACTTCTCCTCCCTCCTCTCTAAATGTAGTCCCCTCTCCTCCATCTAAAGGCAACATTCCTTACCCATTAGTCTCAGAAATTGTCTTAAGCAACAGCCCCAAATGCTGGCTGCCCCCAGCCAAGCATTGGGGCCGCCATCCTGCCTGGCACTGGCTGATGGGCACCTCTGTTGGTTCCATCAGCCAGAGCTCTGCCAAAGGCCCCGCAGTCCCTCTCCCAGGAG

127.0.0.1 - - [13/Nov/2024 15:41:08] "GET /sequence HTTP/1.1" 200 -
